In [ ]:
# Bellow code is tested with the following GPUs: L4

In [1]:
import sys
print(sys.version)

3.11.13 (main, Jun  4 2025, 08:57:29) [GCC 11.4.0]


In [2]:
!nvidia-smi

Wed Aug 13 08:19:19 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA L4                      Off |   00000000:00:03.0 Off |                    0 |
| N/A   47C    P8             12W /   72W |       0MiB /  23034MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [3]:
# Install dependencies
!uv pip install --pre vllm==0.10.1+gptoss --extra-index-url https://wheels.vllm.ai/gpt-oss/ --extra-index-url https://download.pytorch.org/whl/nightly/cu128 --index-strategy unsafe-best-match

Streaming output truncated to the last 5000 lines.
nvidia-cuda-nvrtc-cu12 ------------------------------ 62.52 MiB/83.96 MiB
nvidia-nvshmem-cu12 ------------------------------ 62.54 MiB/118.83 MiB
pytorch-triton ------------------------------ 62.38 MiB/147.53 MiB
nvidia-cufft-cu12 ------------------------------ 65.84 MiB/184.17 MiB
nvidia-cusolver-cu12 ------------------------------ 66.52 MiB/255.11 MiB
nvidia-cusparselt-cu12 ------------------------------ 64.37 MiB/273.89 MiB
nvidia-cusparse-cu12 ------------------------------ 63.38 MiB/274.86 MiB
nvidia-nccl-cu12 ------------------------------ 64.04 MiB/307.42 MiB
nvidia-cublas-cu12 ------------------------------ 60.15 MiB/566.81 MiB
nvidia-cudnn-cu12 ------------------------------ 63.90 MiB/674.02 MiB
vllm       ------------------------------ 271.20 MiB/774.76 MiB
⠙ Preparing packages... (59/72)
ray        ------------------------------ 65.03 MiB/66.83 MiB
nvidia-cuda-nvrtc-cu12 ------------------------------ 62.81 MiB/83.96 MiB
nvi

In [4]:
!nvidia-smi

Wed Aug 13 08:24:42 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA L4                      Off |   00000000:00:03.0 Off |                    0 |
| N/A   41C    P8             12W /   72W |       0MiB /  23034MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
# Doenload model files without serving
# from transformers import AutoModelForCausalLM, AutoTokenizer

# AutoModelForCausalLM.from_pretrained('openai/gpt-oss-20b')
# AutoTokenizer.from_pretrained('openai/gpt-oss-20b')

In [ ]:
!pip install colab-xterm
%load_ext colabxterm

In [ ]:
# ~10 minutes for the first start (include model download)
# ~3 minutes for the second start

# Execute this command in the Terminal
# VLLM_ATTENTION_BACKEND=TRITON_ATTN_VLLM_V1 vllm serve openai/gpt-oss-20b --async-scheduling

In [19]:
from openai import OpenAI

client = OpenAI(
    base_url="http://localhost:8000/v1",
    api_key="EMPTY"
)

In [20]:
system_prompt = """
You are a multiclass multilabel conversation topic classifier.
The only allowed classes are: phone, tv, headphone, internet.
"""

user_content = """
Klientas: Sveiki, ieškau naujo telefono. Mano senas vos pusdienį laiko bateriją.

Asistentas: Supratau! Pradėkime nuo to, kas jums svarbiausia – baterijos veikimo laikas, kamera, našumas ar kas nors kito?

Klientas: Tikrai baterija. Nemėgstu krauti telefono du kartus per dieną.

Asistentas: Puiku, tuomet rekomenduočiau XPhone 14 Max – jis turi 5000 mAh bateriją, kuri įprastai naudojant laiko iki dviejų dienų. Be to, palaiko 45W greitąjį įkrovimą, tad nuo 0% iki 50% įkrausite per mažiau nei 30 minučių.

Klientas: Skamba gerai. O kaip su kamera? Daug fotografuoju socialiniams tinklams.

Asistentas: Šis modelis turi trigubą objektyvų sistemą: 50 MP pagrindinį jutiklį, 12 MP itin plataus kampo ir 10 MP teleobjektyvą. Puikiai fotografuoja prasto apšvietimo sąlygomis, o su AI stabilizacija vaizdo įrašai bus sklandūs.

Klientas: Mhm, o ekranas? Daug žiūriu „Netflix“.

Asistentas: Tai 6,7 colio AMOLED ekranas su 120 Hz atnaujinimo dažniu ir HDR10+ palaikymu – vaizdas bus ryškus ir itin sklandus.

Klientas: O kaip su atmintimi? Nenoriu vis trinti nuotraukų.

Asistentas: Galite rinktis iš 128 GB, 256 GB arba 512 GB. Jei mėgstate laikyti daug vaizdo įrašų, 256 GB būtų optimalus pasirinkimas.

Klientas: Supratau. Kiek kainuoja?

Asistentas: 256 GB versija kainuoja 899 €. Bet šiuo metu turime komplektą – telefonas ir mūsų aukščiausios klasės belaidės ausinės už 949 €.

Klientas: Ausinės?

Asistentas: Taip! XSound Pro belaidės ausinės turi aktyvų triukšmo slopinimą, iki 30 valandų veikimo ir puikiai dera su XPhone garso lustu. Įprastai jos kainuoja 199 €, tad su komplektu sutaupote 150 €. Ir įsivaizduokite „Netflix“ su kino salės garso kokybe.

Klientas: Gundo... Ar būna juodos spalvos?

Asistentas: Taip – juodos, sidabrinės ir riboto leidimo rožinio aukso.

Klientas: Gerai, imu telefoną su ausinėmis.

Asistentas: Puikus pasirinkimas! Paruošiu užsakymą, kad galėtumėte iškart džiaugtis naujuoju įrenginiu.
"""



In [22]:
# ~ 4 seconds for classification
result = client.chat.completions.create(
    model="openai/gpt-oss-20b",
    messages=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_content}
    ]
)

print(result.choices[0].message.content)

phone, headphone


In [18]:
!nvidia-smi

Wed Aug 13 08:42:21 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA L4                      Off |   00000000:00:03.0 Off |                    0 |
| N/A   65C    P0             31W /   72W |   21615MiB /  23034MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----